In [1]:
import numpy as np 
from bs4 import BeautifulSoup #For html parser
import os #to display the  directory contents (not necessary at all)
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import state_union
from sklearn.naive_bayes import MultinomialNB

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df=pd.read_csv('movie_data.csv')
lists = []
stop_words = list(set(stopwords.words("english")))
for i in range(len(df)):
    sent = BeautifulSoup(df['review'][i],"lxml").text
    words = RegexpTokenizer('\w+').tokenize(sent)
    filtered_sent = []
    for m in range(len(words)):
        if words[m] not in stop_words:
            if len(words[m])>=3:
                filtered_sent.append(words[m])
    filteredsent = ' '.join(filtered_sent)
    lists.append((i,filteredsent,df['sentiment'][i]))

In [3]:
lists = pd.DataFrame(lists)
lists.columns = ["index","review","sentiment"]
lists

,index,review,sentiment
0,0,went saw movie last night coaxed friends mine ...,1
1,1,Actor turned director Bill Paxton follows prom...,1
2,2,recreational golfer knowledge sport history pl...,1
3,3,saw film sneak preview delightful The cinemato...,1
4,4,Bill Paxton taken true story 1913 golf open ma...,1
5,5,saw film September 1st 2005 Indianapolis one j...,1
6,6,Maybe reading much wonder much hand Hongsheng ...,1
7,7,felt film many good qualities The cinematograp...,1
8,8,This movie amazing fact real people portray re...,1
9,9,Quitting may much exiting pre ordained identit...,1


In [ ]:
def review_to_words(raw_review):
    #stemmer = SnowballStemmer("english")
    Lemmatizer = WordNetLemmatizer()
    words = raw_review.split()
    tagged = nltk.pos_tag(words)
    stemming_words = [Lemmatizer.lemmatize(w) for w in words]#[stemmer.stem(w) for w in words] 
    newlist = []
    for i in range (len(words)):
        newlist.append(stemming_words[i]+"_"+tagged[i][1])
    return( ' '.join(newlist))
lists['review'] = lists['review'].apply(review_to_words)
lists 

In [ ]:
cv = TfidfVectorizer(min_df = 1, stop_words = 'english')
x_train,x_test,y_train,y_test = train_test_split(lists["review"],lists["sentiment"],test_size = 0.2, random_state=4)
x_traincv = cv.fit_transform(x_train)
y_train = y_train.astype('int')
RFC = RandomForestClassifier()
RFC.fit(x_traincv, y_train) 

In [ ]:
x_testcv = cv.transform(x_test)
pred = RFC.predict(x_testcv)
pred
actual = np.array(y_test)
actual
count = 0;
for i in range (len(pred)):
    if pred[i] == actual[i]:
        count = count+1;
count
len(pred)
print("Accuracy is " + str(100*count/len(pred)) + " %")